In [1]:
import pandas as pd
import requests
import json
import openai
from langchain_openai import ChatOpenAI
import os
from CredentialTo.CredentialToBrokerAPI import CredentialUpstox
from SmartApi.smartConnect import SmartConnect
import settings
from datetime import datetime
import upstox_client 



In [2]:
df_futureOptions = pd.DataFrame()
masterData = pd.read_json("https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz")
# masterData = requests.get(url).json() 
df_futureOptions = pd.DataFrame.from_dict(masterData)

In [3]:
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49906,35.0,...,INDEX,INDEX,BANKNIFTY 54900 PE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
1,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49907,35.0,...,INDEX,INDEX,BANKNIFTY 55000 CE 25 NOV 25,55000.0,1.0,NaN,NaN,NaN,NaN,NaN
2,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49904,35.0,...,INDEX,INDEX,BANKNIFTY 54800 PE 25 NOV 25,54800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49905,35.0,...,INDEX,INDEX,BANKNIFTY 54900 CE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.0,1000.0,NaN,NaN,NaN,NaN,NaN


Convert Expiry to YYYY-MM-DD format

In [4]:
# Convert expiry from epoch (ms) to datetime
df_futureOptions["expiry"] = pd.to_datetime(df_futureOptions["expiry"], unit="ms")

# Convert datetime to string format YYYY-MM-DD
df_futureOptions["expiry"] = df_futureOptions["expiry"].dt.strftime("%Y-%m-%d")


# print(df_futureOptions.dtypes)  # verify expiry is now string
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49906,35.0,...,INDEX,INDEX,BANKNIFTY 54900 PE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
1,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49907,35.0,...,INDEX,INDEX,BANKNIFTY 55000 CE 25 NOV 25,55000.0,1.0,NaN,NaN,NaN,NaN,NaN
2,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49904,35.0,...,INDEX,INDEX,BANKNIFTY 54800 PE 25 NOV 25,54800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49905,35.0,...,INDEX,INDEX,BANKNIFTY 54900 CE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,JPYINR,NSE,2026-03-27,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.0,1000.0,NaN,NaN,NaN,NaN,NaN


Fetch NEarest Expiry date from Today

In [5]:


# Today as Timestamp
today = pd.to_datetime(datetime.today().date())

# Convert expiry_dates to datetime
expiry_dates = pd.to_datetime(df_futureOptions["expiry"], errors="coerce")

# Filter only future expiries
future_expiries = expiry_dates[expiry_dates >= today]

# Find nearest expiry
nearest_expiry = future_expiries.min()

print("Nearest expiry:", nearest_expiry)


Nearest expiry: 2025-09-09 00:00:00


In [6]:
settings.expiry_list_upstox[0]

'2025-09-09'

Filter df_futureOptions BY : ["name"] == "NIFTY" | ["segment"] == "NSE_FO" | expiry

In [7]:
df_futureOptions = df_futureOptions[ (df_futureOptions["name"] == "NIFTY") & (df_futureOptions["segment"] == "NSE_FO")  & (df_futureOptions["expiry"] == settings.expiry_list_upstox[0]) ]
df_futureOptions =  df_futureOptions.reset_index(drop=True)
df_futureOptions

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40700,75.0,...,INDEX,INDEX,NIFTY 26850 PE 09 SEP 25,26850.0,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40650,75.0,...,INDEX,INDEX,NIFTY 25750 PE 09 SEP 25,25750.0,1.0,NaN,NaN,NaN,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40652,75.0,...,INDEX,INDEX,NIFTY 25800 PE 09 SEP 25,25800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40651,75.0,...,INDEX,INDEX,NIFTY 25800 CE 09 SEP 25,25800.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40654,75.0,...,INDEX,INDEX,NIFTY 25850 PE 09 SEP 25,25850.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40478,75.0,...,INDEX,INDEX,NIFTY 23050 PE 09 SEP 25,23050.0,1.0,NaN,NaN,NaN,NaN,NaN
168,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40480,75.0,...,INDEX,INDEX,NIFTY 23100 CE 09 SEP 25,23100.0,1.0,NaN,NaN,NaN,NaN,NaN
169,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40483,75.0,...,INDEX,INDEX,NIFTY 23100 PE 09 SEP 25,23100.0,1.0,NaN,NaN,NaN,NaN,NaN
170,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40465,75.0,...,INDEX,INDEX,NIFTY 23000 PE 09 SEP 25,23000.0,1.0,NaN,NaN,NaN,NaN,NaN


Fetch NIFTY 50 instrument DATA

In [8]:
# Filter NIFTY 50 data
df_nifty50 = df_futureOptions[
    (df_futureOptions["instrument_key"] == "NSE_INDEX|Nifty 50")]

df_nifty50

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit


Fetch NIFTY LTP 

In [9]:
instrument_key='NSE_INDEX|Nifty 50'

df_ltp = pd.DataFrame



from upstox_client.rest import ApiException
from pprint import pprint

# Configure OAuth2 access token for authorization: OAUTH2
configuration = upstox_client.Configuration()
configuration.access_token = CredentialUpstox.ACCESS_TOKEN  
# create an instance of the API class
api_instance = upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

try:
    # Market quotes and instruments - LTP quotes.
    api_response = api_instance.get_ltp(instrument_key=instrument_key)
    
    #  Convert API response to dict
    data_dict = api_response.to_dict()
    
      # Convert to DataFrame
    df_ltp = df_ltp.from_dict(data_dict["data"], orient="index").reset_index()
    df_ltp.rename(columns={"index": "instrument_key"}, inplace=True)
    print(df_ltp)
    
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling MarketQuoteV3Api->get_ltp: %s\n" % e)

       instrument_key  last_price    instrument_token  ltq  volume       cp
0  NSE_INDEX:Nifty 50     24773.0  NSE_INDEX|Nifty 50    0       0  24741.0


Filter By Strike Price

In [10]:

# Extract LTP from response dict
nifty_ltp_value = float(df_ltp["last_price"])

import math

# nifty50_ltp_data = 24740.6

# Round to nearest 50
nearest_strike = round(nifty_ltp_value / 50) * 50  

# Generate 5 strikes around LTP (2 below, 2 above, + the nearest one)
strike_list = [nearest_strike + i * 50 for i in range(-3, 3)]

# Filter DataFrame
df_futureOptions = df_futureOptions[
    df_futureOptions["strike_price"].isin(strike_list)
].reset_index(drop=True)

print("LTP:", nifty_ltp_value)
print("Nearest 5 strikes:", strike_list)


LTP: 24773.0
Nearest 5 strikes: [24600, 24650, 24700, 24750, 24800, 24850]


/var/folders/bm/2sqnclys19zb2dh55g0cx4zc0000gn/T/ipykernel_92032/701431457.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nifty_ltp_value = float(df_ltp["last_price"])


FINAL MASTERDATA

In [11]:
df_futureOptions.head(11)


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40573,75.0,...,INDEX,INDEX,NIFTY 24650 PE 09 SEP 25,24650.0,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40572,75.0,...,INDEX,INDEX,NIFTY 24650 CE 09 SEP 25,24650.0,1.0,NaN,NaN,NaN,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40577,75.0,...,INDEX,INDEX,NIFTY 24700 CE 09 SEP 25,24700.0,1.0,NaN,NaN,NaN,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40579,75.0,...,INDEX,INDEX,NIFTY 24750 CE 09 SEP 25,24750.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40578,75.0,...,INDEX,INDEX,NIFTY 24700 PE 09 SEP 25,24700.0,1.0,NaN,NaN,NaN,NaN,NaN
5,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40580,75.0,...,INDEX,INDEX,NIFTY 24750 PE 09 SEP 25,24750.0,1.0,NaN,NaN,NaN,NaN,NaN
6,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40582,75.0,...,INDEX,INDEX,NIFTY 24800 PE 09 SEP 25,24800.0,1.0,NaN,NaN,NaN,NaN,NaN
7,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40581,75.0,...,INDEX,INDEX,NIFTY 24800 CE 09 SEP 25,24800.0,1.0,NaN,NaN,NaN,NaN,NaN
8,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40571,75.0,...,INDEX,INDEX,NIFTY 24600 PE 09 SEP 25,24600.0,1.0,NaN,NaN,NaN,NaN,NaN
9,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40570,75.0,...,INDEX,INDEX,NIFTY 24600 CE 09 SEP 25,24600.0,1.0,NaN,NaN,NaN,NaN,NaN


In [12]:
df_futureOptions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   weekly             12 non-null     float64
 1   segment            12 non-null     object 
 2   name               12 non-null     object 
 3   exchange           12 non-null     object 
 4   expiry             12 non-null     object 
 5   instrument_type    12 non-null     object 
 6   asset_symbol       12 non-null     object 
 7   underlying_symbol  12 non-null     object 
 8   instrument_key     12 non-null     object 
 9   lot_size           12 non-null     float64
 10  freeze_quantity    12 non-null     float64
 11  exchange_token     12 non-null     int64  
 12  minimum_lot        12 non-null     float64
 13  asset_key          12 non-null     object 
 14  underlying_key     12 non-null     object 
 15  tick_size          12 non-null     float64
 16  asset_type         12 non-nu

CONNECT TO WEBSOCKET LIVE FEED

Fetch All Instrument Keys to subscribe

In [13]:
# ActivateMarketFeed
# token_list = [99926000] is a NIFTY 50 token to get ltp of NIFTY 50 index
token_list = list(set(df_futureOptions['instrument_key'].tolist()))
print(token_list)


['NSE_FO|40570', 'NSE_FO|40584', 'NSE_FO|40577', 'NSE_FO|40578', 'NSE_FO|40583', 'NSE_FO|40571', 'NSE_FO|40582', 'NSE_FO|40581', 'NSE_FO|40573', 'NSE_FO|40580', 'NSE_FO|40579', 'NSE_FO|40572']


Strategy 1 with LIVE FEED

In [22]:
import time
import os, json
import pandas as pd
from openai import OpenAI
import upstox_client
import re, json


# ---- Initialize ----
access_token = CredentialUpstox.ACCESS_TOKEN
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

feed_col = [
    'Token','Open', 'High', 'Low', 'Close', 'Ltp',
    'Vol', 'Oi', 'theta', 'delta', 'gamma', 'vega', 'rho', 'iv', 'vtt'
]
df_feed = pd.DataFrame(columns=feed_col)

_last_eval = 0  # <-- Initialize global timer


# ---- Helpers ----
def build_prompt_from_feed(snapshot_df):
    """
    Convert df_feed snapshot into text for the LLM prompt.
    """
    text = "Here is the latest Nifty50 options feed:\n"
    for _, row in snapshot_df.iterrows():
        text += (
            f"Token: {row['Token']}, "
            f"LTP: {row['Ltp']}, IV: {row['iv']}, OI: {row['Oi']}, "
            f"Greeks: Δ={row['delta']}, Γ={row['gamma']}, Θ={row['theta']}, Vega={row['vega']}\n"
        )
    return text


def ask_llm_for_strategy(snapshot_df, user_objective="Generate an option trading strategy."):
    """
    Ask OpenAI for a strategy using a prompt-based approach.
    """
    feed_text = build_prompt_from_feed(snapshot_df)

    prompt = f"""
    You are an expert Nifty 50 options strategist.
    Your job is to analyze the live option chain feed and propose a clear, structured strategy.

    {feed_text}

    Constraints:
    - Prefer theta income (short premium) strategies.
    - Keep gamma risk controlled.
    - Use 1–4 legs only.
    - Position sizing should be small (not more than 300 qty per leg).
    - Mention rationale clearly.

    Respond in JSON format only with this structure:
    {{
    "strategy_id": "<unique id>",
    "rationale": "<why this strategy>",
    "legs": [
        {{
        "token": "<token>",
        "side": "BUY or SELL",
        "qty": <int>,
        "order_type": "MARKET or LIMIT",
        "price": <float or null>,
        "product": "NRML or MIS"
        }}
    ]
    }}
    """

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0.2,
        messages=[
            {"role": "system", "content": "You are a trading assistant for Nifty 50 options."},
            {"role": "user", "content": prompt}
        ]
    )
    
        # --- Usage inside your ask_llm_for_strategy ---
    raw = resp.choices[0].message.content
    try:
        plan = parse_llm_json(raw)
    except ValueError as e:
        print("LLM returned unparsable JSON:", e)
        plan = None
    return plan


def extract_json_from_text(text: str) -> str:
    """
    Try to pull the first valid JSON object/array substring from text.
    Handles code fences like ```json ... ``` and trailing commentary.
    Returns the json substring (or original text if no braces found).
    """
    if text is None:
        return None
    s = text.strip()

    # remove surrounding triple backticks and optional language hint
    # e.g. ```json\n{...}\n```
    s = re.sub(r"^```(?:json)?\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"\s*```$", "", s)

    # find first JSON opening bracket
    start = None
    for ch in ('{', '['):
        idx = s.find(ch)
        if idx != -1:
            start = idx
            break
    if start is None:
        return s  # no obvious JSON, return as-is

    # find matching bracket using a stack (robust against nested objects)
    pairs = {'{': '}', '[': ']'}
    open_ch = s[start]
    close_ch = pairs[open_ch]
    stack = []
    for i in range(start, len(s)):
        c = s[i]
        if c == open_ch:
            stack.append(close_ch)
        elif c == close_ch:
            stack.pop()
            if not stack:
                # found matching close
                return s[start:i+1]

    # fallback: return substring from first bracket to end
    return s[start:]


def parse_llm_json(raw_text: str):
    """
    Try json.loads(raw_text), otherwise extract JSON substring and try again.
    Returns parsed object or raises ValueError.
    """
    if raw_text is None:
        raise ValueError("Empty response")

    # if the SDK already returned a dict/object, return as-is
    if isinstance(raw_text, (dict, list)):
        return raw_text

    # First try raw
    try:
        return json.loads(raw_text)
    except Exception:
        cleaned = extract_json_from_text(raw_text)
        try:
            return json.loads(cleaned)
        except Exception as e:
            # re-raise with helpful context
            raise ValueError(f"Could not parse JSON. original: {raw_text!r}\ncleaned: {cleaned!r}\nerr: {e}")





# ---- WebSocket Handlers ----
def on_message(message):
    global df_feed, _last_eval

    # Ensure message is dict
    if isinstance(message, str):
        try:
            feeds = json.loads(message)
        except Exception as e:
            print("Failed to parse message:", e, message)
            return
    elif isinstance(message, dict):
        feeds = message.get("feeds", {})  # Upstox WS wraps data under "feeds"
    else:
        print("Unexpected message type:", type(message))
        return

    for token, feed_data in feeds.items():
        marketFF = feed_data.get("fullFeed", {}).get("marketFF", {})

        ltpc = marketFF.get("ltpc", {})
        ohlc_list = marketFF.get("marketOHLC", {}).get("ohlc", [])
        greeks = marketFF.get("optionGreeks", {})

        # Use daily OHLC if available
        ohlc = next((item for item in ohlc_list if item.get("interval") == "1d"), {})

        row = {
            "Token": token,
            "Open": float(ohlc.get("open", 0)),
            "High": float(ohlc.get("high", 0)),
            "Low": float(ohlc.get("low", 0)),
            "Close": float(ohlc.get("close", 0)),
            "Ltp": float(ltpc.get("ltp", 0)),
            "Vol": float(ohlc.get("vol", 0)),
            "Oi": float(marketFF.get("oi", 0)),
            "theta": float(greeks.get("theta", 0)),
            "delta": float(greeks.get("delta", 0)),
            "gamma": float(greeks.get("gamma", 0)),
            "vega": float(greeks.get("vega", 0)),
            "rho": float(greeks.get("rho", 0)),
            "iv": float(marketFF.get("iv", 0)),
            "vtt": float(marketFF.get("vtt", 0)),
        }

        if token in df_feed["Token"].values:
            df_feed.loc[df_feed["Token"] == token, list(row.keys())] = list(row.values())
        else:
            df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)

    # ---- LLM strategy check every 10s ----
    if time.monotonic() - _last_eval > 10 and not df_feed.empty:
        snapshot_df = df_feed.tail(10)
        plan = ask_llm_for_strategy(snapshot_df, "Suggest a neutral option strategy.")
        print("Strategy plan:", plan)
        # pdPlan = pd.read_json(plan)
        # print(pdPlan)
        _last_eval = time.monotonic()

    print(df_feed.tail(3))  # debug


def on_error(error):
    print("Error:", error)


def on_close():
    print("WebSocket closed")


# ---- Connect to Upstox ----
configuration = upstox_client.Configuration()
configuration.access_token = access_token

instrument_key = token_list  # e.g., 'NSE_INDEX|Nifty 50'
mode = 'full'  # full_d30, ltpc, full, etc.

streamer = upstox_client.MarketDataStreamerV3(
    upstox_client.ApiClient(configuration),
    instrument_key,
    mode=mode
)

streamer.on("message", on_message)
streamer.on("error", on_error)
streamer.on("close", on_close)

print("Connecting to Upstox Market Data Stream...")
streamer.connect()
while True:
    time.sleep(1)  # Keep main thread alive


Connecting to Upstox Market Data Stream...
Strategy plan: {'strategy_id': 'Nifty50_Theta_Income_001', 'rationale': 'This strategy aims to generate theta income by selling options with higher implied volatility while controlling gamma risk. The selected legs have manageable delta and gamma values, allowing for a balanced risk profile. The chosen options also have significant open interest, indicating liquidity.', 'legs': [{'token': 'NSE_FO|40584', 'side': 'SELL', 'qty': 300, 'order_type': 'MARKET', 'price': None, 'product': 'NRML'}, {'token': 'NSE_FO|40582', 'side': 'SELL', 'qty': 300, 'order_type': 'MARKET', 'price': None, 'product': 'NRML'}]}
           Token    Open    High     Low   Close     Ltp          Vol  \
9   NSE_FO|40572  180.00  254.85  155.45  166.40  166.40   29564250.0   
10  NSE_FO|40573   28.00   28.00    7.30   17.25   17.25  200734650.0   
11  NSE_FO|40570  220.05  303.00  199.35  211.10  211.10   26708025.0   

           Oi    theta   delta   gamma    vega     rho 

KeyboardInterrupt: 

           Token    Open    High     Low  Close    Ltp          Vol  \
9   NSE_FO|40572  180.00  254.85  155.45  165.9  165.9   29600550.0   
10  NSE_FO|40573   28.00   28.00    7.30   17.7   17.7  201327600.0   
11  NSE_FO|40570  220.05  303.00  199.35  211.1  211.1   26744400.0   

           Oi    theta   delta   gamma    vega     rho        iv          vtt  
9   1497675.0 -38.3334  0.7120  0.0015  4.4576  0.4859  0.174561   29601450.0  
10  8621700.0 -21.1947 -0.2026  0.0018  3.6857 -0.1400  0.116730  201341550.0  
11  2162625.0 -38.6708  0.7598  0.0012  4.0635  0.5176  0.193176   26747475.0  
           Token    Open    High     Low   Close     Ltp          Vol  \
9   NSE_FO|40572  180.00  254.85  155.45  166.25  166.25   29601600.0   
10  NSE_FO|40573   28.00   28.00    7.30   17.70   17.70  201350175.0   
11  NSE_FO|40570  220.05  303.00  199.35  211.35  211.35   26747550.0   

           Oi    theta   delta   gamma    vega     rho        iv          vtt  
9   1497675.0 -38.3334

In [24]:
df_futureOptions.loc[df_futureOptions['instrument_key'] == 'NSE_FO|40582']

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
6,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40582,75.0,...,INDEX,INDEX,NIFTY 24800 PE 09 SEP 25,24800.0,1.0,NaN,NaN,NaN,NaN,NaN


In [25]:
df_feed.head(17)

,Token,Open,High,Low,Close,Ltp,Vol,Oi,theta,delta,gamma,vega,rho,iv,vtt
0,NSE_FO|40583,56.35,87.90,30.50,34.35,34.35,656691975.0,14472900.0,-28.6827,0.3254,0.0022,4.7042,0.2232,0.123749,656692050.0
1,NSE_FO|40584,92.60,111.00,34.10,83.55,83.55,530023950.0,8425875.0,-12.1762,-0.7986,0.0032,3.6725,-0.5525,0.067291,530024475.0
2,NSE_FO|40577,125.35,207.90,114.75,125.60,125.60,135857700.0,5328600.0,-37.5663,0.6466,0.0018,4.8559,0.4419,0.157013,135858300.0
3,NSE_FO|40581,82.00,122.95,50.40,56.35,56.35,688675425.0,15948900.0,-32.8753,0.4453,0.0023,5.1627,0.3052,0.129242,688680450.0
4,NSE_FO|40582,78.20,80.80,20.65,56.20,56.20,857036025.0,18545625.0,-20.8179,-0.5860,0.0036,5.0901,-0.4053,0.083008,857052675.0
5,NSE_FO|40578,42.00,42.00,9.75,25.45,25.45,430667400.0,15099900.0,-23.4959,-0.2908,0.0025,4.4774,-0.2010,0.106506,430676700.0
6,NSE_FO|40579,100.00,163.90,78.60,87.55,87.55,201375900.0,3636000.0,-36.0195,0.5577,0.0021,5.1572,0.3817,0.141754,201375900.0
7,NSE_FO|40571,21.00,21.00,5.80,12.70,12.70,258242550.0,14048325.0,-18.9702,-0.1459,0.0014,2.9896,-0.1008,0.128784,258266325.0
8,NSE_FO|40580,56.00,57.00,13.65,37.60,37.60,454604775.0,11679750.0,-23.9814,-0.4159,0.0031,5.0955,-0.2875,0.095520,454605300.0
9,NSE_FO|40572,180.00,254.85,155.45,167.50,167.50,29608575.0,1497675.0,-38.3348,0.7120,0.0015,4.4571,0.4858,0.174561,29609025.0


In [17]:
# import os, json
# from openai import OpenAI

# # Create client
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# def build_prompt_from_feed(snapshot_df):
#     """
#     Convert df_feed snapshot into text for the LLM prompt.
#     """
#     text = "Here is the latest Nifty50 options feed:\n"
#     for _, row in snapshot_df.iterrows():
#         text += (
#             f"Token: {row['Token']}, "
#             f"LTP: {row['Ltp']}, IV: {row['iv']}, OI: {row['Oi']}, "
#             f"Greeks: Δ={row['delta']}, Γ={row['gamma']}, Θ={row['theta']}, Vega={row['vega']}\n"
#         )
#     return text

# def ask_llm_for_strategy(snapshot_df, user_objective="Generate an option trading strategy."):
#     """
#     Ask OpenAI for a strategy using a prompt-based approach.
#     """
#     feed_text = build_prompt_from_feed(snapshot_df)
    
#     prompt = f"""
#     You are an expert Nifty 50 options strategist.
#     Your job is to analyze the live option chain feed and propose a clear, structured strategy.

#     {feed_text}

#     Constraints:
#     - Prefer theta income (short premium) strategies.
#     - Keep gamma risk controlled.
#     - Use 1–4 legs only.
#     - Position sizing should be small (not more than 300 qty per leg).
#     - Mention rationale clearly.

#     Respond in JSON format only with this structure:
#     {{
#     "strategy_id": "<unique id>",
#     "rationale": "<why this strategy>",
#     "legs": [
#         {{
#         "token": "<token>",
#         "side": "BUY or SELL",
#         "qty": <int>,
#         "order_type": "MARKET or LIMIT",
#         "price": <float or null>,
#         "product": "NRML or MIS"
#         }}
#     ]
#     }}
#     """

#     resp = client.chat.completions.create(
#         model="gpt-5",
#         temperature=0.2,
#         messages=[
#             {"role":"system","content":"You are a trading assistant for Nifty 50 options."},
#             {"role":"user","content":prompt}
#         ]
#     )

#     raw = resp.choices[0].message.content
#     try:
#         plan = json.loads(raw)   # ensure it’s JSON
#     except:
#         print("LLM did not return valid JSON:", raw)
#         return None
#     return plan


ADD LLM ( MAGIC)

In [18]:
# from langchain_openai import ChatOpenAI
# from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# os.environ["OPENAI_API_KEY"] = ""



# df = df_feed

# # LLM setup
# llm = ChatOpenAI(model="gpt-4", temperature=0)

# # Create agent
# agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True )

# # Ask questions
# print(agent.run(""" you are a stock market expert.You have all the historical knowledge of NSE stock market. 
#                  DO : which token has highest oi?  
#                 """))